# __PROBLEMA BUS DI SUPPORTO COVID__



---
## Testo del problema 

Produrre un notebook con il codice che, 
dato un numero di autobus aggiuntivi, 
determina le sequenze di porzioni di viaggio da affidare a ciascun veicolo e massimizza il numero di passeggeri raccolti. 
Una porzione di viaggio può essere assegnata ad al massimo un veicolo. 
Analizzare il valore della funzione obiettivo al variare del numero di autobus aggiuntivi.


### Pacchetti aggiuntivi utilizzati


* [pandas](https://pandas.pydata.org/docs/index.html)
* [openpyxl](https://openpyxl.readthedocs.io/en/stable/)
* [numpy](https://numpy.org/)
* [datetime](https://docs.python.org/3/library/datetime.html)
* [networkx](https://networkx.org/documentation/stable/)
* [matplotlib.pyplot](https://matplotlib.org/stable/users/index.html)
* [seaborn](https://seaborn.pydata.org/introduction.html)

    Importiamo le librerie in questione e per semplicità le denominiamo con delle sigle.  
    Questo ci consente di richiamare i moduli e le sue funzioni citando solamente la sigla

In [ ]:
import pandas as pd
import openpyxl as opx
import numpy as np
from datetime import date, datetime, time, timedelta
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sbn

### Scelta file

Vista l'ambiguità delle date nel file fornito abbiamo deciso di dare all'utente la possibilità di scegliere se usare lo stesso oppure un file modificato da noi contenente le stesse informazioni condensate nello stesso giorno. Le soluzioni saranno ovviamente differenti a seconda del file scelto.

In [ ]:
risp = False
while (risp == False):
    ans = input('Quale file si vuole utilizzare?\n\n1 = File Originale\n\n2 = File modificato con porzioni di viaggio tutte nella stessa data\n\nDigitare il numero dell\'opzione   ')
    if (ans == '1' or ans == '2'):
        risp = True

### Inserire su python i parametri presenti sul file selezionato
   
   > [file di dati](./file%20di%20dati.xlsx)
   
   > [file di dati singolo giorno](./file%20di%20dati%20singolo%20giorno.xlsx)
   

Per fare questo, ci serviamo delle librerie _PANDAS_ e _OPENPYXL_ la quale consentono l'importazione di tabelle da excel e in seguito l'importazione di questi in una struttura di dati con la funzione `DataFrame()`

La scelta di utilizzare `DataFrame()` è stata dettata da una migliore possibilità di gestione dei dati e visualizzazione

Con l'utilizzo di _Openpyxl_, importiamo le tabelle del foglio excel selezionato.

In particolare: 

>__tratte__ sarà una lista di liste contenente i dati presenti sul foglio "porzioni di viaggio" del file 
 
>__tempi__ sarà una seconda lista di liste contenente i dati presenti nel foglio "tempi di percorrenza a vuoto" dello stesso file
 
Una volta importati i dati, generiamo dei DataFrame ovvero delle strutture dati ordinate contenenti le medesime informazioni

In [ ]:
if ans == '1':
    file = 'file di dati.xlsx'
else:
    file = 'file di dati singolo giorno.xlsx'

xlsx   = opx.load_workbook(filename = file, data_only=True)
tratte = xlsx['porzioni di viaggio']
tempi  = xlsx['tempi di percorrenza a vuoto']

df_tempi    = pd.DataFrame(np.array([[i.value for i in j] for j in  tempi['A2':'Q18']]))  
df_porzioni = pd.DataFrame(np.array([[i.value for i in j] for j in tratte['A2':'F158']])) 

xlsx.close()

### L'utente sceglie il numero massimo di autobus aggiuntivi
 
 Ogni volta che viene eseguito il programma, esso richiederà di inserire:

>`max_bus` : Numero massimo di autobus di supporto a disposizione del comune

Conoscendo il numero massimo di autobus presenti nel deposito, calcoliamo il numero di passeggeri raccolti da ___k___ autobus, per ogni _k_ che va da _0_ a _max_bus_.

In [ ]:
max_bus = int(input("\nINSERIRE il numero di autobus a disposizione del comune \nIn seguito premere INVIO:\n\n\n"))

K = set(range(1, max_bus + 1))

## Formattiamo correttamente le tabelle importate

### Formattazione tabella df_tempi

Abbiamo corretto la formattazione "grezza" data dall'importazione delle tabelle.

Per fare questo abbiamo:
    
1. creato un tipo _Series_ contenente i nomi delle fermate :   __ser_t__

In [ ]:
ser_t = df_tempi.iloc[0:17, 0]

2. Sostituito gli indici vecchi delle tabelle con `ser_t`

In [ ]:
df_tempi.columns = ser_t
df_tempi.index = ser_t

3. Abbiamo eliminato la riga e la colonna delle intestazioni vecchie dalla tabella

In [ ]:
df_tempi.columns = df_tempi.columns.fillna('drop')
df_tempi.index = df_tempi.index.fillna('drop')
df_tempi.drop('drop', inplace = True)
df_tempi.drop('drop', axis = 1, inplace = True)

### Formattazione tabella df_porzioni

Allo stesso modo abbiamo corretto la formattazione "grezza" anche del DataFrame delle porzioni di viaggio a vuoto.

In [ ]:
ser_p = df_porzioni.iloc[0, 0:6]
df_porzioni.columns = ser_p
df_porzioni.drop( 0, inplace = True)

### Ulteriore correzione alla tabella

La nostra intenzione è quella di utilizzare i nomi delle stazioni per accedere ai dati delle due tabelle.
Per fare questo e per confrontare le percorrenze, abbiamo la necessità che le stringhe contenenti i __NOMI DELLE STAZIONI__ siano uguali

Per fare questo:
    
* abbiamo corretto i nomi presenti nelle colonne 'C' e 'D' della tabella __df_porzioni__ 

* abbiamo inoltre riscritto i nomi delle stazioni nella tabella __df_tempi__


In [ ]:
for i in range (0, 156):
    for j in range (2, 4):
        if 'medag' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "medaglie d'oro"
        if ('stazione' in df_porzioni.iat[i, j] and df_porzioni.iat[i, j] != 'autostazione'):
            df_porzioni.iat[i, j] = "stazione"
        if 'kenn' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "kennedy"
        if 'medoro' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "corti di medoro"
        if 'pontegra' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "pontegradella itis"
        if 'smm' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "smm"
        else: continue
            

idx_list = df_tempi.index.tolist()
idx = pd.Series(idx_list)
idx[5] = "medaglie d'oro"
idx = idx.replace({'stazione fs':'stazione', 'piazzale dante':'p. dante', 'santa maria maddalena': 'smm'})
df_tempi.index = idx
df_tempi.columns = idx


# SVOLGIMENTO DEL PROBLEMA

Una volta corrette le importazioni delle due tabelle, passiamo alla risoluzione del problema.

### Matrice di compatibilità
Abbiamo generato una matrice __C__ definita di compatibilità:

$
c_{ij} =\begin{cases} 1 & \mbox{se le tratte i-j sono compatibili} \\ 0 & \mbox{Altrimenti }\end{cases}
$

Per valutare la compatibilità tra le tratte _i_ e _j_ dobbiamo verificare che, sommando all'orario di fine della tratta _i_ il tempo di percorrenza a vuoto da _i_ a _j_, si ottenga un orario antecedente all'orario di partenza della tratta _j_.

In formule:

$
T_{\mbox{fine lavoro bus supporto}} + T_{\mbox{percorrenza a vuoto per arrivare alla nuova stazione}} \leq T_{\mbox{inizio nuovo lavoro di supporto}}
$

In [ ]:
c = np.zeros((156, 156))
I = set(range(1, 157))
for i in I:
    for j in I:
        if df_porzioni.at[i, "final stop"] == df_porzioni.at[j, "initial stop"]:
            continue
        elif df_tempi.at[df_porzioni.at[i, "final stop"], df_porzioni.at[j, "initial stop"]] is None:
                delta = df_tempi.at[df_porzioni.at[j, "initial stop"], df_porzioni.at[i, "final stop"]]
        else:
                delta = df_tempi.at[df_porzioni.at[i, "final stop"], df_porzioni.at[j, "initial stop"]]
        
        if (df_porzioni.at[i, "final time"] + timedelta(minutes = delta) <= df_porzioni.at[j, "initial time"]):
            c[i-1, j-1] = 1
        else :
            c[i-1, j-1] = 0   

C = pd.DataFrame(c)

### Passeggeri Extra

Ora definiamo il _DataFrame_ __df_extra__ che presenta il numero di passeggeri che hanno bisogno di un autobus di supporto per ogni tratta

In [ ]:
extra_pass = df_porzioni["extra passengers"]
df_extra = extra_pass.copy() 
tratte_bus = df_porzioni [['initial stop', 'final stop']]
df_tratte = tratte_bus.copy()

## Soluzione grafica

Abbiamo affrontato il problema graficamente.

Abbiamo generato un grafo avente i due nodi principali _POZZO_ e _SORGENTE_.<br>
Ad essi abbiamo aggiunto due serie di nodi che rappresentano le tratte raffigurate nella tabella __df_porzioni__<br>
I nodi della prima serie che chiameremo <ins>"nodo 1"</ins> sarà lato sorgente e collegata ad essa<br>
I nodi della seconda serie <ins>"nodo 2"</ins> saranno collegata ai nodi primari delle tratte con cui la tratta che rappresentano è compatibile e al pozzo  

__Abbiamo separato la stessa tratta in due nodi distinti perchè altrimenti più di un autobus si recherebbe nello stesso nodo per raccogliere i passeggeri, pur non essendo efettivamente assegnato alla tratta in questione.__





<img src="./immagine_lab.PNG" width ="700">
<center>Per agevolare la comprensione della topologia del grafo abbiamo allegato un'immagine esemplificativa</center>

<br><br>

### Costruzione del Grafo

Definiamo il grafo direzionale __G__ con la funzione `DiGraph()` della libreria _Networkx_. 
<br>

Con la funzione `add_edge()` creiamo gli archi che collegano i nodi primari con i loro secondari.<br>In realtà, dato che i nodi $u_i$ e $v_i$ non esistono ancora, la stessa funzione li genera.<br>
Gli archi in questione hanno:
* Peso 0
* Capacità 1 

La capacità unitaria di questi archi garantisce che solamente un'unità di flusso(autobus) attraversi l'arco, poichè questi archi sono gli unici in uscita nei nodi primari, garantisce anche che sia una sola unità di flusso a passare per il nodo primario 

In [ ]:
G = nx.DiGraph()

for i in I:
    u_i = '{}. {} -> {} nodo1'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    v_i = '{}. {} -> {} nodo2'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    G.add_edge(u_of_edge = u_i, v_of_edge = v_i, weight = 0, capacity = 1)

Il grafo ora inizializzato può essere così popolato:

Se $c_{ij}$ = 1, viene aggiunto un arco avente

* Peso pari all'opposto del numero di passeggeri extra nella tratta in questione 
* Capacità 1 


Questi archi collegano, per ogni tratta, il nodo secondario al nodo primario di tutte le tratte ad esso compatibili.


La motivazione legata al costo negativo di questi archi è data dal fatto che con questo metodo è possibile risolvere il grafo come un problema di<br> __flusso di costo minimo__



In [ ]:
for i in I:
    for j in I:
        if c[i-1, j-1] == 1:
            u_comp = '{}. {} -> {} nodo2'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
            v_comp = '{}. {} -> {} nodo1'.format(j, df_tratte.at[j, 'initial stop'],df_tratte.at[j, 'final stop'])
            G.add_edge(u_of_edge = u_comp, v_of_edge = v_comp, weight = -df_extra[j] , capacity = 1)
            


Aggiungiamo in seguito i nodi:  
* __s__  Sorgente  
* __t__  Pozzo

In seguito aggiungiamo tutti gli archi che dalla _sorgente_ vanno verso i nodi primari e tutti gli archi che dai nodi secondari vanno al _pozzo_

In [ ]:
G.add_node('s')
G.add_node('t')
for i in I:
    u_i = '{}. {} -> {} nodo1'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    v_i = '{}. {} -> {} nodo2'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    G.add_edge(u_of_edge = 's', v_of_edge = u_i, weight = -df_extra[i], capacity = 1)
    G.add_edge(u_of_edge = v_i, v_of_edge = 't', weight = 0, capacity = 1)

---

### Soluzione al problema

Per ogni iterazione viene aggiunto l'arco che collega il _pozzo_ con la _sorgente_ avente:
* Peso 0
* Capacità __k__

Questo arco permette di rispettare il vincolo sul numero di autobus.

Avremmo potuto in modo equivalente aggiungere delle etichette ai nodi pozzo e sorgente, rispettivamente _-k_ e _k_

Una volta completata la struttura viene calcolato il __FLUSSO DI COSTO MINIMO__ in funzione di _k_.

__Al raggiungimento del numero di autobus necessari a raccogliere tutti i passeggeri possibili il programma termina.__

In [ ]:
flow_costs_k = []

k = 0
incremento = True
while (incremento and k < max_bus):
    G.add_edge(u_of_edge = 't', v_of_edge = 's', weight = 0, capacity = k)
    flowCost, flowDict = nx.algorithms.flow.network_simplex(G, capacity ='capacity', weight ='weight')
    flow_costs_k.append((-1)*flowCost)
    
    if (flow_costs_k[k] == flow_costs_k[k-1] and k > 1):
        flow_max = flow_costs_k[k-1]
        incremento = False
    k = k + 1
    
if(incremento == False):
    for i in range(3):
        flow_costs_k.append(flow_max)

flow_edges = []

for u, v in G.edges():
    G[u][v]['weight'] = -G[u][v]['weight']
    if flowDict[u][v] != 0:
        flow_edges.append(('%s'%u,'%s'%v))


### Generazione istogramma 
Il problema conta e produce un istogramma che mostra il numero di passeggeri raccolti in funzione del numero _k_ di autobus impiegati.

Grazie all'utilizzo della libreria _Seaborn_, con la funzione `catplot()` generiamo un istogramma

Sull'asse delle x c'è il valore di __k__  
Sull'asse delle y invece il numero di passeggeri trasportati




In [ ]:
ser_k = pd.Series(flow_costs_k)

ser_k = pd.DataFrame(ser_k)
ser_k = ser_k.transpose()
plot = sbn.catplot(data =ser_k, kind ="bar", aspect = 2, height =10, palette ="crest")

plot.savefig('Istogramma.png')


Per praticità e migliore visualizzazione, abbiamo esportato la figura in file:

>[Istogramma.png](./Istogramma.png)  
    
    Cliccando il link si può così vedere il risultato finale